In [1]:
from math import sqrt

In [2]:
import numpy as np

In [3]:
from geometry import triangulate, transform, wall_pieces

In [4]:
from helpers import make_mesh, write_mesh

In [5]:
EIGHTH = [
    [(0, 0, 2), (0, 1, 1), (1, 1, 1)],
    [(0, 2, 1), (0, 1, 1), (1, 1, 1)],
    [(0, 2, 1), (0, 2, 0), (1, 1, 1)],
    [(2, 2, 0), (0, 2, 0), (1, 1, 1)],
]   

In [6]:
Z_SCALE = sqrt(2)

In [7]:
QUARTER = EIGHTH + transform(EIGHTH, lambda x, y, z: (y, x, z))

In [8]:
WALL = 0.4
FLOOR = 0.6

In [9]:
HEIGHT = 2 * Z_SCALE + FLOOR

In [10]:
SIZE = 2 + WALL

In [11]:
INNER_QUARTER = transform(QUARTER, lambda x, y, z: (x, y, HEIGHT - Z_SCALE * z))

In [12]:
INNER_QUARTER

[[(0, 0, 0.6000000000000001),
  (0, 1, 2.0142135623730955),
  (1, 1, 2.0142135623730955)],
 [(0, 2, 2.0142135623730955),
  (0, 1, 2.0142135623730955),
  (1, 1, 2.0142135623730955)],
 [(0, 2, 2.0142135623730955),
  (0, 2, 3.4284271247461904),
  (1, 1, 2.0142135623730955)],
 [(2, 2, 3.4284271247461904),
  (0, 2, 3.4284271247461904),
  (1, 1, 2.0142135623730955)],
 [(0, 0, 0.6000000000000001),
  (1, 0, 2.0142135623730955),
  (1, 1, 2.0142135623730955)],
 [(2, 0, 2.0142135623730955),
  (1, 0, 2.0142135623730955),
  (1, 1, 2.0142135623730955)],
 [(2, 0, 2.0142135623730955),
  (2, 0, 3.4284271247461904),
  (1, 1, 2.0142135623730955)],
 [(2, 2, 3.4284271247461904),
  (2, 0, 3.4284271247461904),
  (1, 1, 2.0142135623730955)]]

In [13]:
QUARTER_FRONT_TRI_RAW = [
    [(0, 1, 1), (0, 0, 2), (0, 1, 2)],
    [(0, 1, 1), (0, 2, 2), (0, 1, 2)],
    [(0, 1, 1), (0, 2, 1), (0, 2, 2)],
]

QUARTER_FRONT_TRI = transform(QUARTER_FRONT_TRI_RAW, lambda x, y, z: (x, y, HEIGHT - Z_SCALE * z))

FRONT_BOTTOM_RECT = [
    (0, 0, 0), (0, 0, FLOOR), (0, 2, FLOOR), (0, 2, 0)
]

QUARTER_FRONT = QUARTER_FRONT_TRI + list(triangulate(FRONT_BOTTOM_RECT))

In [14]:
MOULD_QUARTER = INNER_QUARTER + QUARTER_FRONT

In [15]:
RIM_QUARTER = list(triangulate(
    [(0, 2, HEIGHT), (0, SIZE, HEIGHT), (SIZE, SIZE, HEIGHT), (2, 2, HEIGHT)]
))

TOP_RIM = (
    RIM_QUARTER
    + transform(RIM_QUARTER, lambda x, y, z: (y, x, z))
    + transform(RIM_QUARTER, lambda x, y, z: (x, -y, z))
    + transform(RIM_QUARTER, lambda x, y, z: (y, -x, z))
)

In [16]:
FLOOR_SQUARE = list(triangulate(
    [(0, 0, 0), (0, 2, 0), (2, 2, 0), (2, 0, 0)]
))

FLOOR_RIM = transform(TOP_RIM, lambda x, y, z: (x, y, 0))

FLOOR_ALL = FLOOR_RIM + FLOOR_SQUARE

In [17]:
HEIGHTS = [0, FLOOR, FLOOR + Z_SCALE, FLOOR + 2 * Z_SCALE]

In [18]:
LEFT_OUTER_PIECES = [
    [(0, 2), (0, SIZE)],
    [(0, SIZE), (SIZE, SIZE)],
    [(SIZE, SIZE), (SIZE, 0)],
]

OUTER_PIECES = LEFT_OUTER_PIECES + [[(x, -y) for x, y in piece] for piece in LEFT_OUTER_PIECES]

WALL_OUTER = list(tri for piece in OUTER_PIECES for tri in wall_pieces(*piece, heights=HEIGHTS))

In [19]:
WRAPPER = FLOOR_ALL + WALL_OUTER + TOP_RIM

In [20]:
OUTER_MOULD = MOULD_QUARTER + WRAPPER

In [22]:
DIVISION = (
    transform(QUARTER_FRONT, lambda x, y, z: (y, x, z))
    + list(wall_pieces((2, 0), (2, -2), HEIGHTS))
    + list(wall_pieces((2, -2), (0, -2), HEIGHTS))
)

In [23]:
PIECE_4 = OUTER_MOULD + DIVISION

In [24]:
MM_SCALE = 15

In [25]:
write_mesh(PIECE_4, MM_SCALE, 'output/piece_4.stl')

In [26]:
FILLER_SQUARE = transform(FLOOR_SQUARE, lambda x, y, z: (x, -y, z))
FILLER_MOULD_QUARTER = transform(MOULD_QUARTER, lambda x, y, z: (x, -y, z))

FILLER = FILLER_SQUARE + FILLER_MOULD_QUARTER

In [27]:
PIECE_5 = FILLER + DIVISION

In [28]:
write_mesh(PIECE_5, MM_SCALE, 'output/piece_5.stl')

In [29]:
PIECE_1 = OUTER_MOULD + FILLER

In [30]:
write_mesh(PIECE_1, MM_SCALE, 'output/piece_1.stl')